In [1]:
import requests
from config import req_headers, base_url, focus_league, local_host, local_name, local_port, local_user
from creds import local_db_password
import psycopg2
import time

In [2]:
conn = psycopg2.connect(dbname=local_name, user =local_user, host=local_host, password=local_db_password, port =local_port)
cur = conn.cursor()

In [3]:
endpoint = 'players'
url = base_url + endpoint
season = 2023

In [4]:
team_list = []
fetch_teams_sql = """select id from ffl.teams"""
cur.execute(fetch_teams_sql)
for i in cur.fetchall():
    team_list.append(i[0])

In [5]:
player_insert_sql = """
insert into ffl.players (
    id,
    current_team_id,
    season,
    position,
    name_first,
    name_last,
    name_easy,
    height,
    weight,
    date_of_birth
)
values
(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
on conflict do nothing
"""

In [6]:
def parser(data=None):
    player_id = int(data['player']['id'])
    team_id = int(data['statistics'][0]['team']['id'])
    player_season = season
    position = data['statistics'][0]['games']['position']
    name_first = data['player']['firstname']
    name_last = data['player']['lastname']
    name_easy = data['player']['name']
    height = data['player']['height']
    weight = data['player']['weight']
    date_of_birth = data['player']['birth']['date']

    return player_id, team_id, player_season, position, name_first, name_last, name_easy, height, weight, date_of_birth

In [7]:
def fetch_response(params=None, page = None):
    if page == None:
        req = requests.get(url, params=params, headers=req_headers)

        return req.json()
    else:
        params['page'] = page
        req = requests.get(url, params=params, headers=req_headers)

        return req.json()

def load_responses(response_data=None):
    for player in response_data['response']:
        cur.execute(player_insert_sql, (parser(player)))
    # conn.commit()

In [8]:
for team in team_list:
    time.sleep(30)
    # print(f'starting new team team id {team}')
    call_params = {
    'league':focus_league,
    'season':season,
    'team':team
    }

    response = fetch_response(call_params)
    load_responses(response)
    conn.commit()
    paging_needed = True
    while paging_needed == True:
        paging_current = response['paging']['current']
        paging_total = response['paging']['total']
        if paging_current == paging_total:
            paging_needed = False
            # print('paging not needed')
        elif paging_current != paging_total:
            next_page = paging_current + 1
            response = fetch_response(call_params, next_page)
            load_responses(response)
            conn.commit()
            # print(f'completed paging. page #{next_page}')

starting new team team id 33
completed paging. page #2
completed paging. page #3
completed paging. page #4
paging not needed
starting new team team id 34
completed paging. page #2
completed paging. page #3
paging not needed
starting new team team id 35
completed paging. page #2
completed paging. page #3
paging not needed
starting new team team id 36
paging not needed
starting new team team id 39
paging not needed
starting new team team id 40
paging not needed
starting new team team id 42
paging not needed
starting new team team id 44
paging not needed
starting new team team id 45
paging not needed
starting new team team id 47
paging not needed
starting new team team id 48
paging not needed
starting new team team id 49
paging not needed
starting new team team id 50
paging not needed
starting new team team id 51
paging not needed
starting new team team id 52
paging not needed
starting new team team id 55
paging not needed
starting new team team id 62
paging not needed
starting new team t